# 填充和步幅

In [1]:
import torch
from torch import nn
import d2l.torch as d2l

### 填充 padding

假设输入的形状为 $n_h \times n_w$ , 卷积核形状为 $k_h \times k_w$ , 则输出的形状为

$$
(n_h - k_h + 1) \times (n_w - k_w + 1)
$$

如果对输入进行填充, 使用 $p_h$ 行进行填充, $p_w$ 列进行填充, 大约一半在上下, 一半在左右,则输出形状为

$$
(n_h - k_h + p_h + 1) \times (n_w -k_w +p_w +1)
$$

卷积核的 $size$ 通常来说都是奇数, 这样的好处是能够在上下左右填充相同数量的列, 另外, 卷积核的 $size$ 一般来说就是 $1,3,5$ , 小的卷积核可以理解为 $MLP$ 中使用更少的 $unit$ 但是更深的网络

一般来说设置 $p_h = k_h -1$ 和 $p_w = k_w -1$

定义一个函数用于计算卷积层, 对输入进行计算后在进行升维或降维

In [2]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1,1) + X.shape) # (1, 1) + (m * n) = (1, 1, m, n)
    Y = conv2d(X) # 计算
    return Y.reshape(Y.shape[2:]) # 省略批量大小和通道

$padding$ 在 $torch$ 里就是单独的一边添多少

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2,1))
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 步幅 stride

设垂直步幅为 $s_h$ , 水平步幅为 $s_w$ , 输出的形状为

$$
\lfloor \dfrac{(n_h-k_h+p_h+s_h)}{s_h}\rfloor \times \lceil \dfrac{(n_w-k_w+p_w+s_w)}{s_w} \rceil
$$

这是因为填充了之后的矩阵 $size=(n_h \times p_h)$ , 再减去 $k_h, k_w$ 之后, 就是空出来能够用来跳跃的长度了, 但是 $k_h, k_h$ 那里还会有一个能放下的矩形, 所以最后的答案就是上面的输出形状

因为设置了 $p_h=k_h-1$ 和 $p_w = k_w-1$ , 所以输出的形状变成了

$$
\lfloor \dfrac{(n_h+s_h-1)}{s_h}\rfloor \times \lceil \dfrac{(n_w+s_w-1)}{s_w} \rceil
$$

进一步, 一般来说是 $\dfrac{n_h}{s_h}\times\dfrac{n_w}{s_w}$

In [8]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

再随便看一个例子, 但是在通常情况下我们不会这样做, 我们会设置 $kernel\_size$ 和 $stride$ 都为一个数字, 而不是一个 $tuple$

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])